#### 1. feature selction pipeline - cat - ordinal encoding
#### 2.final pipeline - cat - OneHotEncde

In [26]:
from  warnings import filterwarnings
filterwarnings ("ignore")

#### step 1 -Data ingestion 

In [68]:
import pandas as pd 
df = pd .read_csv("Cars93.csv",na_values=["", "NA"], keep_default_na=False)
df.head()

,id,Manufacturer,Model,Type,Min.Price,Price,Max.Price,MPG.city,MPG.highway,AirBags,...,Passengers,Length,Wheelbase,Width,Turn.circle,Rear.seat.room,Luggage.room,Weight,Origin,Make
0,1,Acura,Integra,Small,12.9,15.9,18.8,25,31,None,...,5,177,102,68,37,26.5,11.0,2705,non-USA,Acura Integra
1,2,Acura,Legend,Midsize,29.2,33.9,38.7,18,25,Driver & Passenger,...,5,195,115,71,38,30.0,15.0,3560,non-USA,Acura Legend
2,3,Audi,90,Compact,25.9,29.1,32.3,20,26,Driver only,...,5,180,102,67,37,28.0,14.0,3375,non-USA,Audi 90
3,4,Audi,100,Midsize,30.8,37.7,44.6,19,26,NaN,...,6,193,106,70,37,31.0,17.0,3405,non-USA,Audi 100
4,5,BMW,535i,Midsize,23.7,30.0,36.2,22,30,Driver only,...,4,186,109,69,39,27.0,13.0,3640,non-USA,BMW 535i


#### step 2- perform basic data quality checks

In [28]:
df.shape

(93, 28)

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 93 entries, 0 to 92
Data columns (total 28 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   id                  93 non-null     int64  
 1   Manufacturer        93 non-null     object 
 2   Model               93 non-null     object 
 3   Type                93 non-null     object 
 4   Min.Price           93 non-null     float64
 5   Price               93 non-null     float64
 6   Max.Price           93 non-null     float64
 7   MPG.city            93 non-null     int64  
 8   MPG.highway         93 non-null     int64  
 9   AirBags             89 non-null     object 
 10  DriveTrain          93 non-null     object 
 11  Cylinders           93 non-null     object 
 12  EngineSize          93 non-null     float64
 13  Horsepower          93 non-null     int64  
 14  RPM                 93 non-null     int64  
 15  Rev.per.mile        93 non-null     int64  
 16  Man.trans.

In [29]:
m = df.isna().sum()
m[m>0]

AirBags            4
Rear.seat.room     2
Luggage.room      11
dtype: int64

#### step 3 - seprate x and y (weight)

In [30]:
x = df.drop(columns=["id", "Weight"])
y = df[["Weight"]]

In [31]:
df.duplicated().sum()

np.int64(0)

In [32]:
x.head()

,Manufacturer,Model,Type,Min.Price,Price,Max.Price,MPG.city,MPG.highway,AirBags,DriveTrain,...,Fuel.tank.capacity,Passengers,Length,Wheelbase,Width,Turn.circle,Rear.seat.room,Luggage.room,Origin,Make
0,Acura,Integra,Small,12.9,15.9,18.8,25,31,None,Front,...,13.2,5,177,102,68,37,26.5,11.0,non-USA,Acura Integra
1,Acura,Legend,Midsize,29.2,33.9,38.7,18,25,Driver & Passenger,Front,...,18.0,5,195,115,71,38,30.0,15.0,non-USA,Acura Legend
2,Audi,90,Compact,25.9,29.1,32.3,20,26,Driver only,Front,...,16.9,5,180,102,67,37,28.0,14.0,non-USA,Audi 90
3,Audi,100,Midsize,30.8,37.7,44.6,19,26,NaN,Front,...,21.1,6,193,106,70,37,31.0,17.0,non-USA,Audi 100
4,BMW,535i,Midsize,23.7,30.0,36.2,22,30,Driver only,Rear,...,21.1,4,186,109,69,39,27.0,13.0,non-USA,BMW 535i


In [33]:
y.head()

,Weight
0,2705
1,3560
2,3375
3,3405
4,3640


#### step 4- forward feature selection 

In [69]:
cat =list (x.columns[x.dtypes == "object"])
con = list(x.columns[x.dtypes != "object"])

In [35]:
cat

['Manufacturer',
 'Model',
 'Type',
 'AirBags',
 'DriveTrain',
 'Cylinders',
 'Man.trans.avail',
 'Origin',
 'Make']

In [36]:
con

['Min.Price',
 'Price',
 'Max.Price',
 'MPG.city',
 'MPG.highway',
 'EngineSize',
 'Horsepower',
 'RPM',
 'Rev.per.mile',
 'Fuel.tank.capacity',
 'Passengers',
 'Length',
 'Wheelbase',
 'Width',
 'Turn.circle',
 'Rear.seat.room',
 'Luggage.room']

In [37]:
from sklearn.pipeline import make_pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OrdinalEncoder,StandardScaler
from sklearn.compose import ColumnTransformer

In [38]:
num_pipe1 = make_pipeline(
    SimpleImputer(strategy="mean"),
    StandardScaler()
)

In [66]:
pre1 = ColumnTransformer(
    [
        ("num", num_pipe1, con),
        ("cat", num_pipe1, cat)

    ]

) .set_output(transform="pandas")

    
    


In [67]:
X_pre = pre1.fit_transform(x)
X_pre.head()

ValueError: Cannot use mean strategy with non-numeric data:
could not convert string to float: 'Acura'

In [20]:
from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import SequentialFeatureSelector

In [44]:
model1 = LinearRegression()
sel = SequentialFeatureSelector(model1, n_features_to_select="auto",direction="forward")
sel.fit_transform(x_pre,y)
sel_cols = sel.get_feature_names_out()
sel_cols

NameError: name 'x_pre' is not defined

In [45]:
len(sel_cols)

NameError: name 'sel_cols' is not defined

In [ ]:
sel_cols

In [ ]:
sel_cols[0].split("__")

In [ ]:
sel_cols[0].split("__")[1]

In [46]:
imp_cols = [col.split("__")[1] for col in sel_cols]
imp_cols

NameError: name 'sel_cols' is not defined

In [ ]:
x_sel = x[imp_cols]
x_sel.head()


#### create a final pipeline for x

##### cat - oneHotEncoder

In [47]:
cat_sel = list(x_sel.columns[x_sel.dtypes == "object"])
con_sel = list(x_sel.columns[x_sel.dtypes != "object"])

NameError: name 'x_sel' is not defined

In [ ]:
cat_sel

In [ ]:
con_sel

In [ ]:
from sklearn.preprocessing import OneHotEncoder

In [51]:
num_pipe2 = make_pipeline(
    SimpleImputer(strategy="mean"),
    StandardScaler()
)

In [53]:
cat_pipe2 = make_pipeline(
    SimpleImputer(strategy ="most_frequent"),
    OneHotEncoder(handle_unknown="ignore", sparse_output =False)

)

NameError: name 'OneHotEncoder' is not defined

In [54]:
pre2 =ColumnTransformer(
    [
        ("num", num_pipe2, con_sel),
        ("cat", cat_pipe2, cat_sel),
        ("cat", cat_pipe2, cat_sel)
    ]
).set_output(transform="pandas")


NameError: name 'con_sel' is not defined

In [55]:
x_sel.head()

NameError: name 'x_sel' is not defined

In [ ]:
x_sel_pre = pre2.fit_transform(x_sel)
x_sel_pre.head()

#### step 5 Train Test Spilt

In [56]:
from sklearn.model_selection import train_test_split
xtrain, xtest, ytrain, ytest = train_test_split(x_sel, Y, test_size=0.2, random_state=21)

NameError: name 'x_sel' is not defined

In [57]:
xtrain.head()

NameError: name 'xtrain' is not defined

In [ ]:
ytrain.head()

In [ ]:
xtest.head()

In [ ]:
ytest.head()

In [ ]:
xtrain.shape

In [ ]:
xtest.shape

### Step 6 -Bulid Model

In [58]:
final_model = LinearRegression()
final_model.fit(xtrain, ytrain)



NameError: name 'xtrain' is not defined

In [ ]:
final_model.intercept_

In [ ]:
final_model.coef_

#### Step 7- Evaluate model 

In [ ]:
final_model.score(xtrain,ytrain)

In [ ]:
final_model.score(xtest,ytest)

In [59]:
from sklearn.metrics import(
    root_mean_squared_error,
    mean_absolute_error,
    mean_absolute_percentage_error,
    r2_score
)

In [60]:
def evaluate_model(model, x,y):
    ypred = model.predict(x)
    rmse = root_mean_squared_error(y,ypred)
    mae = mean_absolute_error(y,ypred)
    mape = mean_absolute_percentage_error(y,ypred)
    r2 =r2_score(y,ypred)
    print(F"RMSE :{rmse: .2f}")
    print(F"MAE :{mae: .2f}")
    print(F"MAPE :{mape: .4f}")
    print(F"R2 score :{r2: .4f}")
    
                 

In [ ]:
print("train Results :")
evaluate_model(final_model,xtrain,ytrain)

In [ ]:
print("Test Results : ")
evaluate_model(final_model,xtest,ytest)

In [61]:
xnew = pd.read_csv("sample.csv")
xnew

,Manufacturer,Model,Type,Min.Price,Price,Max.Price,MPG.city,MPG.highway,AirBags,DriveTrain,...,Fuel.tank.capacity,Passengers,Length,Wheelbase,Width,Turn.circle,Rear.seat.room,Luggage.room,Origin,Make
0,Audi,100,Midsize,30.8,37.7,44.6,19,26,NaN,Front,...,15.0,6,190,106,65,37,31.0,17.0,non-USA,Audi 100
1,Pontiac,Sunbird,Compact,9.4,11.1,12.8,23,31,NaN,Front,...,15.2,5,181,101,66,39,25.0,13.0,USA,Pontiac Sunbird
2,Chevrolet,Lumina,Midsize,13.4,15.9,18.4,21,29,NaN,Front,...,16.5,6,198,108,71,40,28.5,16.0,USA,Chevrolet Lumina
3,Mazda,RX-7,Sporty,32.5,32.5,32.5,17,25,Driver only,Rear,...,20.0,2,169,96,69,37,NaN,NaN,non-USA,Mazda RX-7
4,Volkswagen,Fox,Small,8.7,9.1,9.5,25,33,NaN,Front,...,12.4,4,163,93,63,34,26.0,10.0,non-USA,Volkswagen Fox


In [62]:
pre2

NameError: name 'pre2' is not defined

In [ ]:
xnew_pre = pre2.transform(xnew)
xnew_pre

In [63]:
preds = final_model.predict(xnew_pre)
preds

NameError: name 'xnew_pre' is not defined

In [ ]:
xnew["weight_pred"] = preds

In [64]:
xnew = xnew.round(2)
xnew

,Manufacturer,Model,Type,Min.Price,Price,Max.Price,MPG.city,MPG.highway,AirBags,DriveTrain,...,Fuel.tank.capacity,Passengers,Length,Wheelbase,Width,Turn.circle,Rear.seat.room,Luggage.room,Origin,Make
0,Audi,100,Midsize,30.8,37.7,44.6,19,26,NaN,Front,...,15.0,6,190,106,65,37,31.0,17.0,non-USA,Audi 100
1,Pontiac,Sunbird,Compact,9.4,11.1,12.8,23,31,NaN,Front,...,15.2,5,181,101,66,39,25.0,13.0,USA,Pontiac Sunbird
2,Chevrolet,Lumina,Midsize,13.4,15.9,18.4,21,29,NaN,Front,...,16.5,6,198,108,71,40,28.5,16.0,USA,Chevrolet Lumina
3,Mazda,RX-7,Sporty,32.5,32.5,32.5,17,25,Driver only,Rear,...,20.0,2,169,96,69,37,NaN,NaN,non-USA,Mazda RX-7
4,Volkswagen,Fox,Small,8.7,9.1,9.5,25,33,NaN,Front,...,12.4,4,163,93,63,34,26.0,10.0,non-USA,Volkswagen Fox


In [65]:
xnew.to_csv("forwardResults.csv", index=False)